In [3]:
!pip install pandas
!pip install sentence-transformers
!pip install accelerate
!pip install -U bitsandbytes
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 MB 133.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14/14 [datasets]/14 [datasets]yeballs]


In [ ]:
import pandas as pd
df = pd.read_csv('final.csv')
laliste=[]
laliste_labels=[]
for elt in df['label']:
    laliste_labels.append(elt)
for elt in df['text']:
    laliste.append(elt)

In [13]:
laliste_labels=[]
for elt in train['intent']:
    laliste_labels.append(elt)

In [1]:
from sentence_transformers import SentenceTransformer
import torch

# Define arguments for 4-bit quantization
model_kwargs = {
    "device_map": "auto",         # Automatically map layers to devices (GPU/CPU)
    "torch_dtype": torch.bfloat16,  # Recommended dtype for Qwen models
    "load_in_4bit": True,           # Enable 4-bit quantization
}

# The model card also recommends enabling flash_attention_2 for better performance
# You may need to install it: pip install flash-attn
# Set use_flash_attention_2=True if available
embedding_model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-8B",
    model_kwargs=model_kwargs,
    trust_remote_code=True, # Often required for custom model architectures
    # use_flash_attention_2=True, # Uncomment if flash-attn is installed
)

/home/ubuntu/synthdata1/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.17s/it]


In [4]:
from datasets import load_dataset
ds = load_dataset("hrithikpiyush/acl-arc")
ds.set_format("pandas")
train=ds["train"][:]
validation=ds["validation"][:]
test=ds["test"][:]

Generating test split: 100%|██████████| 139/139 [00:00<00:00, 2064.16 examples/s]


In [6]:
documents = laliste[:10]

len(documents)

10

In [7]:
synth_data = torch.from_numpy(embedding_model.encode(documents))

In [8]:

#documents = laliste

synth_data = torch.from_numpy(embedding_model.encode(train['cleaned_cite_text'].tolist()))
validation_data = torch.from_numpy(embedding_model.encode(validation['cleaned_cite_text'].tolist()))
test_data = torch.from_numpy(embedding_model.encode(test['cleaned_cite_text'].tolist()))

In [15]:
import torch

import torch.nn as nn

class MLPClassifier(nn.Module):
    def __init__(self, input_dim=4096, hidden_dim=128, num_classes=6):
        super(MLPClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        return self.model(x)
mymodel = MLPClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mymodel.parameters(), lr=0.001)

In [16]:
from torch.utils.data import TensorDataset, DataLoader
X_train = synth_data
y_train = torch.tensor(laliste_labels)

batch_size = 32
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [17]:
num_epochs = 10

for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()           # ✅ Reset gradients from previous step
        outputs = mymodel(X_batch)        # Forward pass
        loss = criterion(outputs, y_batch)
        loss.backward()                 # Compute gradients
        optimizer.step()                # Update weights


In [18]:
from sklearn.metrics import classification_report

In [19]:
with torch.no_grad():
    validation_outputs = mymodel(validation_data)
    test_outputs = mymodel(test_data)

In [20]:
torch.argmax(validation_outputs,axis=1)

tensor([0, 2, 0, 2, 0, 0, 1, 1, 0, 0, 0, 0, 2, 1, 1, 2, 0, 2, 1, 2, 0, 0, 0, 0,
        0, 1, 0, 2, 0, 0, 0, 0, 0, 3, 3, 2, 0, 1, 1, 1, 0, 2, 1, 4, 2, 2, 1, 1,
        0, 3, 1, 2, 5, 1, 3, 1, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, 2,
        2, 3, 0, 0, 0, 2, 0, 0, 2, 1, 0, 0, 0, 2, 0, 5, 2, 1, 0, 2, 0, 2, 2, 0,
        0, 1, 0, 4, 2, 1, 0, 2, 4, 0, 5, 0, 5, 1, 1, 0, 2, 2])

In [22]:
print(classification_report(validation['intent'], torch.argmax(validation_outputs, axis=1)))

              precision    recall  f1-score   support

           0       0.82      0.71      0.76        59
           1       0.62      0.71      0.67        21
           2       0.63      0.81      0.71        21
           3       0.20      0.25      0.22         4
           4       0.33      0.20      0.25         5
           5       0.75      0.75      0.75         4

    accuracy                           0.69       114
   macro avg       0.56      0.57      0.56       114
weighted avg       0.71      0.69      0.69       114

